In [1]:
!ls /opt/spark/data

Fire_Department_Calls_For_Service__2016__20240816.csv


In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession
             .builder
             #.enableHiveSupport()
             .master("spark://spark-master:7077")
             .config("spark.sql.warehouse.dir", "/opt/spark/spark-warehouse")
             .getOrCreate()
             )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/26 08:53:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df=spark.read.format("csv").option("header", True).option("inferSchema", True).load("/opt/spark/data/Fire_Department_Calls_For_Service__2016__20240816.csv")


In [ ]:
df.rdd.getNumPartitions()

10

In [6]:
for cols in df.columns:
    df=df.withColumnRenamed(cols, cols.lower().replace(" ","_"))

In [8]:
df.printSchema()

root
 |-- call_number: integer (nullable = true)
 |-- unit_id: string (nullable = true)
 |-- incident_number: integer (nullable = true)
 |-- call_date: string (nullable = true)
 |-- watch_date: string (nullable = true)
 |-- received_dttm: string (nullable = true)
 |-- entry_dttm: string (nullable = true)
 |-- dispatch_dttm: string (nullable = true)
 |-- response_dttm: string (nullable = true)
 |-- on_scene_dttm: string (nullable = true)
 |-- transport_dttm: string (nullable = true)
 |-- hospital_dttm: string (nullable = true)
 |-- call_final_disposition: string (nullable = true)
 |-- available_dttm: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- zipcode_of_incident: double (nullable = true)
 |-- battalion: string (nullable = true)
 |-- station_area: string (nullable = true)
 |-- box: string (nullable = true)
 |-- original_priority: string (nullable = true)
 |-- priority: string (nullable = true)
 |-- final_priority: integer (nul

In [11]:
df3=df.select("unit_type").drop_duplicates()
df3.count()

12

In [12]:
df3.rdd.getNumPartitions()

1

In [10]:
df2=df.groupBy("unit_type").count()

In [11]:
df2.rdd.getNumPartitions()

1

In [18]:
df2.show()

+--------------+------+
|     unit_type| count|
+--------------+------+
|       AIRPORT|   710|
|         MEDIC|842164|
|         CHIEF|198516|
|  RESCUE SQUAD| 38290|
|           BLS|  9753|
|RESCUE CAPTAIN| 91032|
|         TRUCK|265292|
| INVESTIGATION|  2374|
|            CP| 30597|
|        ENGINE|922851|
|       SUPPORT| 83609|
|       PRIVATE|263791|
+--------------+------+



In [15]:
df.cache()

25/04/25 13:50:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[call_number: int, unit_id: string, incident_number: int, call_date: string, watch_date: string, received_dttm: string, entry_dttm: string, dispatch_dttm: string, response_dttm: string, on_scene_dttm: string, transport_dttm: string, hospital_dttm: string, call_final_disposition: string, available_dttm: string, address: string, city: string, zipcode_of_incident: double, battalion: string, station_area: string, box: string, original_priority: string, priority: string, final_priority: int, als_unit: boolean, call_type_group: string, number_of_alarms: int, unit_type: string, unit_sequence_in_call_dispatch: int, fire_prevention_district: string, supervisor_district: string, neighborhooods_-_analysis_boundaries: string, rowid: string]

In [16]:
df.count()

2748979

In [13]:
spark.conf.set("spark.sql.adaptive.enabled", "false")

In [16]:
df2=df.groupBy("unit_type").count()
df2.show()

+--------------+------+
|     unit_type| count|
+--------------+------+
|       AIRPORT|   710|
|         MEDIC|842164|
|         CHIEF|198516|
|  RESCUE SQUAD| 38290|
|           BLS|  9753|
|RESCUE CAPTAIN| 91032|
|         TRUCK|265292|
| INVESTIGATION|  2374|
|            CP| 30597|
|        ENGINE|922851|
|       SUPPORT| 83609|
|       PRIVATE|263791|
+--------------+------+



In [15]:
df2.rdd.getNumPartitions()

200

In [12]:
spark.conf.get("spark.sql.shuffle.partitions")

'200'

In [20]:
spark.conf.set("spark.sql.shuffle.partitions", 20)